<a href="https://colab.research.google.com/github/filall/main/blob/main/xboyyy_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import requests
from bs4 import BeautifulSoup
from translate import Translator
import re
# Replace 'YOUR_TOKEN' with your actual Bot token received from BotFather
TOKEN = '7076261734:AAH4-Xs0v2IaF87lD73W3PyuMXTaCgX_VjU'
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('أرسل لي رابط منتج من على AliExpress وسأحصل لك على العنوان بالعربية (للجزء بعد العلامة "|") والصورة الرئيسية.')
def get_product_details(url: str) -> dict:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract title
            title_tag = soup.find('meta', property='og:title')
            title = title_tag.get('content') if title_tag else "لم يتم العثور على عنوان المنتج."

            # Replace "MAD" with "ثمن المنتج بالدرهم"
            title = re.sub(r'\bMAD', 'ثمن المنتج بالدرهم', title)

            # Split title at "|"
            title_parts = title.split("|")
            title_before_pipe = title_parts[0].strip()

            # Extract the price, discount, and remaining title
            price_discount_title = title_parts[1].strip()
            price_discount_title_parts = re.split(r'(\d+%OFF)', price_discount_title)
            price = price_discount_title_parts[0].strip()
            discount = price_discount_title_parts[1].strip()
            remaining_title = title_parts[2].strip()

            # Reconstruct the title with the desired format
            new_title = f"{price}\n{discount} | {remaining_title}"


            # Extract image
            image_tag = soup.find('meta', property='og:image')
            image_url = image_tag.get('content') if image_tag else "لم يتم العثور على الصورة."
            return {
                'title_before_pipe': title_before_pipe,

                'image_url': image_url,
                'url': url
            }
        else:
            return {
                "title_before_pipe": "",
                "new_title_arabic": "فشل في استرداد الصفحة. الرجاء التحقق من الرابط.",
                "image_url": "",
                'url': url
            }
    except Exception as e:
        return {
            'title_before_pipe': "",
            'new_title_arabic': f'حدث خطأ: {e}',
            'image_url': '',
            'url': url
        }

def handle_message(update: Update, context: CallbackContext) -> None:
    text = update.message.text
    if 'aliexpress.com' in text and re.match(r'https?://\S+', text):
        details = get_product_details(text)
        caption = f'{details["title_before_pipe"]}\n'

        if details["new_title_arabic"]:
            caption += details["new_title_arabic"]

        caption += f'\nرابط الشراء: {details["url"]}'

        if details["image_url"]:
            update.message.reply_photo(photo=details["image_url"], caption=caption)
        else:
            update.message.reply_text(caption)
    else:
        update.message.reply_text('الرجاء إرسال رابط صحيح لمنتج على AliExpress.')


def main() -> None:
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))
    updater.start_polling()
    updater.idle()
if __name__ == '__main__':
    main()


